# Promptomatix Basic Usage

This notebook demonstrates the three core workflows of Promptomatix:
1. **Prompt Optimization** - Convert a raw input into an optimized prompt
2. **Feedback Generation** - Generate feedback on the optimized prompt
3. **Optimization with Feedback** - Improve the prompt based on feedback

## Prerequisites

Make sure you have:
1. Installed Promptomatix: `./install.sh`
2. Activated the environment: `source Promptomatix_env/bin/activate`
3. Set your API keys: `export OPENAI_API_KEY="your_key"`

---

## Setup and Imports

In [1]:
# Import required libraries
import os
import re
import sys
import json
import importlib
from dotenv import load_dotenv
from promptomatix import main

importlib.reload(main)

# Add the src directory to Python path
sys.path.append('../src')

# Import Promptomatix functions
from promptomatix.main import process_input, generate_feedback, optimize_with_feedback, save_feedback

# Load environment variables
load_dotenv()

# Get API key from environment
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("⚠️  Please set your OPENAI_API_KEY environment variable")
    print("   export OPENAI_API_KEY='your_api_key_here'")
else:
    print("✅ API key loaded successfully")

/Users/rithesh.murthy/Workspace/promptomatic/promptomatic_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ API key loaded successfully


## 1. Prompt Optimization

Let's start with a simple example: optimizing a prompt for text classification.

In [2]:
# Define the configuration for prompt optimization
initial_prompt = "guess the sentiment of the given vague tweet and classify it into one single class - positive, negative, or neutral"

config = {
    "raw_input": initial_prompt, # the prompt that needs to be optimized
    "model_name": "gpt-3.5-turbo",
    "model_api_key": api_key,
    "model_provider": "openai",
    "backend": "simple_meta_prompt",
    "synthetic_data_size": 10,  # Size of synthetic examples
    "task_type": "classification"
}

In [3]:
# Run the optimization
print('='*80)
result = process_input(**config)

print('='*80)
print(f"📝 Optimized Prompt: {result['result']}")
print(f"🆔 Session ID: {result['session_id']}")
print(f"💰 Cost: ${result['metrics']['cost']:.4f}")
print(f"⏱️ Time: {result['metrics']['time_taken']:.2f} seconds")

🚀 Starting Promptomatic optimization...
🎯 Running optimization...
📊 Generating 10 synthetic samples...
  ✓ Batch 1: 8 samples (8/10)
  ✓ Batch 2: 2 samples (10/10)
✅ Generated 10 synthetic samples
🔧 Evaluating initial prompt...
  Initial score: 1.0000
🔧 Generating optimized prompt...
📊 Evaluating optimized prompt...
  Optimized score: 1.0000
✅ Prompt optimization complete!
✅ Optimization completed successfully!
📝 Optimized Prompt: Classify the sentiment of the given ambiguous tweet as positive, negative, or neutral.
🆔 Session ID: 1751069700.159286
💰 Cost: $0.2037
⏱️ Time: 102.05 seconds


In [4]:
# Let's look at the synthetic data generated
print("📊 Synthetic Data Generated:")
print('='*80)
for i, example in enumerate(result['synthetic_data'], 1):
    # Handle input fields (can be single string or list)
    if isinstance(result['input_fields'], list):
        input_values = []
        for field in result['input_fields']:
            input_values.append(f"{field}: {example.get(field, 'N/A')}")
        input_display = " | ".join(input_values)
    else:
        input_display = example.get(result['input_fields'], 'N/A')
    
    # Handle output fields (can be single string or list)
    if isinstance(result['output_fields'], list):
        output_values = []
        for field in result['output_fields']:
            output_values.append(f"{field}: {example.get(field, 'N/A')}")
        output_display = " | ".join(output_values)
    else:
        output_display = example.get(result['output_fields'], 'N/A')
    
    print(f"{i}. Input: {input_display}")
    print(f"   Output: {output_display}")
    print()

📊 Synthetic Data Generated:
1. Input: tweet: Thrilled to find out my flight got delayed for the third time today. 🙃 #patience
   Output: sentiment: negative

2. Input: tweet: The new software update is just what I needed to slow down my phone. 🙄 #techwoes
   Output: sentiment: negative

3. Input: tweet: I suppose the presentation was informative, though it felt like it dragged on forever. 😐
   Output: sentiment: neutral

4. Input: tweet: Finally finished the marathon, and now I can't feel my legs. 😅 #worthit
   Output: sentiment: positive

5. Input: tweet: The concert was amazing, except for the part where it started raining. 🙂 #memorable
   Output: sentiment: positive

6. Input: tweet: I love how my coffee machine decided to break on a Monday morning. 😒 #greatstart
   Output: sentiment: negative

7. Input: tweet: I can't believe I won the lottery, but lost the ticket. 😭 #unbelievable
   Output: sentiment: negative

8. Input: tweet: The presentation went well, but I can't shake off the

## 2. Feedback Generation

Now let's generate feedback on our optimized prompt to see how it can be improved. <br>
Note: This is a bit slow because we are using a reasoning model (Like OAI's o3) for generating feedback

In [5]:
# Generate feedback on the optimized prompt
print("🔄 Generating feedback...")

feedback_result = generate_feedback(
    optimized_prompt=result['result'],
    input_fields=result['input_fields'],
    output_fields=result['output_fields'],
    model_name="gpt-3.5-turbo",
    model_api_key=api_key,
    synthetic_data=result['synthetic_data'],
    session_id=result['session_id']
)

print("✅ Feedback generated!")
print(f"\n📋 Comprehensive Feedback:")
print(feedback_result['comprehensive_feedback'])

🔄 Generating feedback...
🧠 Generating feedback...
🔄 Processing synthetic data samples...


🧠 Processing sample 10/10: 100%|██████████████████████████████| 10/10 [01:12<00:00,  7.28s/sample]


[DEBUG] Feedback generation time_taken (ms): 75.509159
✅ Feedback generation completed!
✅ Feedback generated!

📋 Comprehensive Feedback:
```json
{
  "primary_feedback": [
    "Add a rule such as: 'If a tweet expresses any noticeable negative emotion (e.g., regret, worry, disappointment, sarcasm) alongside positive statements, classify it as Negative rather than Neutral.'",
    "Provide bullet definitions, e.g. 'Negative: expresses worry, sadness, anger, self-criticism, fear, regret, or any other negative emotion—even if partially positive.' Include 2–3 short tweet examples for each class."
  ],
  "secondary_feedback": [
    "Specify desired casing, e.g., 'Use lowercase for the sentiment label.'",
    "Treat hashtags or emojis that convey emotions (e.g., #perfectionistproblems, #fail) as sentiment clues."
  ]
}
```


In [6]:
# Look at individual feedback for specific examples
print("🔍 Individual Feedback Examples:")
for i, feedback in enumerate(feedback_result['individual_feedbacks'][:3], 1):
    print(f"\nExample {i}:")
    print(f"  Input: {feedback['user_input']}")
    print(f"  Expected: {feedback['expected_output']}")
    print(f"  AI Output: {feedback['ai_output']}")
    print(f"  Feedback: {feedback['feedback'][:200]}...")

🔍 Individual Feedback Examples:

Example 1:
  Input: Thrilled to find out my flight got delayed for the third time today. 🙃 #patience
  Expected: negative
  AI Output: Neutral
  Feedback: {
  "summary": "The AI produced a neutral label because the prompt is too terse: it neither defines what constitutes each sentiment class nor mentions that sarcasm, irony, or emojis can invert the lit...

Example 2:
  Input: The new software update is just what I needed to slow down my phone. 🙄 #techwoes
  Expected: negative
  AI Output: Negative
  Feedback: {
  "summary": "Mismatch arises from output formatting, not sentiment classification. The prompt never specifies the required case, so the AI’s capitalized \"Negative\" is a reasonable interpretation....

Example 3:
  Input: I suppose the presentation was informative, though it felt like it dragged on forever. 😐
  Expected: neutral
  AI Output: Neutral
  Feedback: ```json
{
  "summary": "The only mismatch is capitalization: the AI returned \"Neutr

## 3. Optimization with Feedback

Now let's improve our prompt based on the feedback we received.

In [7]:
# Filter for critial feedback only

def filter_critical_feedback(feedback_result):
    try:
        # Get the comprehensive feedback string
        comprehensive_feedback = feedback_result['comprehensive_feedback']
        
        json_match = re.search(r'```json\s*(.*?)\s*```', comprehensive_feedback, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
        else:
            json_str = comprehensive_feedback

        feedback_data = json.loads(json_str)
        
        # Extract primary feedback
        primary_feedback = feedback_data.get('primary_feedback', [])
        return " ".join(primary_feedback)
    except (json.JSONDecodeError, KeyError, AttributeError) as e:
        print(f"Error extracting primary feedback: {e}")
        return ""
    
critial_feedback =  filter_critical_feedback(feedback_result)


In [8]:
print(f"Final feedback: {critial_feedback}")

Final feedback: Add a rule such as: 'If a tweet expresses any noticeable negative emotion (e.g., regret, worry, disappointment, sarcasm) alongside positive statements, classify it as Negative rather than Neutral.' Provide bullet definitions, e.g. 'Negative: expresses worry, sadness, anger, self-criticism, fear, regret, or any other negative emotion—even if partially positive.' Include 2–3 short tweet examples for each class.


In [9]:
# Save the feedback to the session & then initiate the optimization

# Save feedback
feedback_result = save_feedback(
    text=result['result'],  # The prompt text you want to provide feedback on
    start_offset=0,  # Starting position of the text you're providing feedback on
    end_offset=len(result['result']),  # Ending position
    feedback=critial_feedback,  # Your actual feedback
    prompt_id=result['session_id']  # The session ID from the initial process_input call
)

# Optimize the prompt using the feedback
improved_result = optimize_with_feedback(result['session_id'])

print("\n"+"="*20+"REPORT"+ "="*20)
print(f"📝 Original Prompt:")
print(result['result'])
print(f"\n📝 Improved Prompt:")
print(improved_result['result'])
print(f"\n💰 Additional Cost: ${improved_result['metrics']['cost']:.4f}")
print(f"⏱️  Additional Time: {improved_result['metrics']['time_taken']:.2f} seconds")

💾 Saving feedback...
✅ Feedback saved successfully!
🔄 Optimizing with feedback...
📊 Generating 10 synthetic samples...
  ✓ Batch 1: 10 samples (10/10)
✅ Generated 10 synthetic samples
🔧 Evaluating initial prompt...
  Initial score: 1.0000
🔧 Generating optimized prompt...
📊 Evaluating optimized prompt...
  Optimized score: 1.0000
✅ Prompt optimization complete!
✅ Feedback optimization completed!

====================REPORT====================
📝 Original Prompt:
Classify the sentiment of the given ambiguous tweet as positive, negative, or neutral.

📝 Improved Prompt:
Classify the sentiment of the provided tweet as positive, negative, or neutral:

- **Positive**: Expresses emotions like happiness, satisfaction, or excitement.
  - Examples: 
    - "Just got promoted at work, feeling on top of the world!"
    - "Loving the new restaurant in town, the food is amazing!"
    - "Had a fantastic day at the beach with friends!"

- **Negative**: Contains any negative emotion (e.g., worry, sadness,

In [10]:
# Let's look at the synthetic data generated - post feedback optimization
print("📊 Synthetic Data Generated:")
print('='*80)
for i, example in enumerate(improved_result['synthetic_data'], 1):
    # Handle input fields (can be single string or list)
    if isinstance(improved_result['input_fields'], list):
        input_values = []
        for field in improved_result['input_fields']:
            input_values.append(f"{field}: {example.get(field, 'N/A')}")
        input_display = " | ".join(input_values)
    else:
        input_display = example.get(improved_result['input_fields'], 'N/A')
    
    # Handle output fields (can be single string or list)
    if isinstance(improved_result['output_fields'], list):
        output_values = []
        for field in improved_result['output_fields']:
            output_values.append(f"{field}: {example.get(field, 'N/A')}")
        output_display = " | ".join(output_values)
    else:
        output_display = example.get(improved_result['output_fields'], 'N/A')
    
    print(f"{i}. Input: {input_display}")
    print(f"   Output: {output_display}")
    print()

📊 Synthetic Data Generated:
1. Input: tweet: Just completed a marathon, my legs are sore but the sense of achievement is incredible!
   Output: sentiment: positive

2. Input: tweet: Received a compliment on my presentation, but I felt it was undeserved.
   Output: sentiment: negative

3. Input: tweet: The stock market closed today with mixed results, some gains and some losses.
   Output: sentiment: neutral

4. Input: tweet: Explored a new hiking trail today, the views were breathtaking!
   Output: sentiment: positive

5. Input: tweet: Had a productive day at work, but the constant interruptions were frustrating.
   Output: sentiment: negative

6. Input: tweet: The new software update was released today.
   Output: sentiment: neutral

7. Input: tweet: Spent the afternoon gardening, the flowers are blooming beautifully!
   Output: sentiment: positive

8. Input: tweet: The concert was amazing, but the sound system had issues.
   Output: sentiment: negative

9. Input: tweet: Cooked dinner

## Summary

We've successfully demonstrated the three core workflows:

1. **✅ Prompt Optimization**: Converted a raw input into an optimized prompt
2. **✅ Feedback Generation**: Generated detailed feedback on the prompt
3. **✅ Optimization with Feedback**: Improved the prompt based on feedback

### Key Takeaways:

- **Session Management**: Each optimization creates a session that can be referenced later
- **Synthetic Data**: The system automatically generates training examples
- **Cost Tracking**: All operations track API costs and execution time
- **Iterative Improvement**: Feedback can be used to continuously improve prompts

### Next Steps:

- Try different task types (summarization, translation, etc.)
- Experiment with different models (GPT-4, Claude, etc.)
- Explore the advanced features in other notebooks
- Use the CLI interface for batch processing

---

**Ready to explore more?** Check out:
- `02_prompt_optimization.ipynb` - Advanced optimization techniques
- `03_metrics_evaluation.ipynb` - Performance analysis
- `04_advanced_features.ipynb` - Custom configurations